In [1]:
# Import necessary packages:
import cobra
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy
from datetime import datetime, timedelta
from scipy.interpolate import RegularGridInterpolator
import netCDF4
from IPython.display import clear_output

# Author-written functions:
import DaySolve, NightSolve, fluxCalc, Light, Extras
import MergeDaySolve, MergeNightSolve, MergefluxCalc, MergeLoadModel

In [2]:
# Initialize a dictionary to hold all inputs to the class.
Inputs = {}

# User defined inputs here!!! Names will be more descriptive in this user-facing section.
Inputs['Year'] = 2024
Inputs['DayNumber'] = 0 # Daynumber 0 is a flag for not using DAYNUM/YY mode.
Inputs['Month'] = 3 # Month 0 is a flag for not using MM/DD/YYYY mode.
Inputs['Day'] = 1 # Day 0 is a flag for not using MM/DD/YYYY mode.
Inputs['Hour'] = 12 # 24-hour time
Inputs['Minute'] = 0
Inputs['Second'] = 0
Inputs['Interval'] = 0 # Defaults to zero. Do not change, interval will be handed in dFBA.
Inputs['Latitude'] = 75 # Bottom of Arctic Circle
Inputs['Longitude'] = 0 # Prime Meridian
Inputs['Timezone'] = 0 # We are at the prime meridian. (This will be automatically calculated.)
Inputs['Pressure'] = 1013 # DEFAULT: 1013 mb = 1 atmosphere
Inputs['Temperature'] = 0 # DEFAULT: 10 deg C (arctic will be much colder)

Inputs['Turbidity'] = 0.084
Inputs['Water Vapor'] = 1.4164
Inputs['Ozone'] = 0.3438 # GET CITATIONS FOR THESE FROM ORIGINAL SMARTS2
Inputs['Albedo'] = 0.1 # Average albedo of ocean surface

# Input wavelength-dependent data tables.
Inputs['WVL-ETR'] = np.loadtxt('.\\Data\\WVL-ETR.csv',delimiter=',',dtype='float64', encoding='utf-8-sig')
Inputs['WVL-ABS'] = np.loadtxt('.\\Data\\WVL-ABS.csv',delimiter=',',dtype='float64', encoding='utf-8-sig')

# Create an object that will store data for wavelength-dependent light transmission through the atmosphere.
ICalc = Light.TotSolEng(Inputs)
# Calculate the total incident extraterrestrial radiation (ETR) based on date, time and location.
ICalc.CalcTotSolarEng(Inputs)
# Calculate wavelength-dependent light transmission based on ETR spectrum.
ICalc.CalcSolarSpec(Inputs)

# Calculate wavelength-dependent light absorption/scattering by the ocean at a certain depth.
ICalc.waterabs(0)
ICalc.DTOTphoto = np.asarray(ICalc.DTOTphoto).squeeze()

Inputs['Itot'] = scipy.integrate.simpson(ICalc.DTOTphoto,ICalc.allWVLphoto)

# Integrate only the photoactive region of the incident spectrum to get total photoactive light intensity.
Inputs['Iphoto'] = Extras.par(ICalc.DTOTphoto,ICalc.allWVLphoto)

start = str(Inputs['Month']).zfill(2) + "/" + str(Inputs['Day']).zfill(2) + "/" + str(Inputs['Year']) + " " + str(Inputs['Hour'])
start_date = datetime.strptime(start, "%m/%d/%Y %H")

In [7]:
# Calculate the saturation concentration of carbon dioxide (in mM).
ML, refmodel, IYSi, INSi = LoadModel()

Henry = henry(12)
CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
O2sat = Henry[1]*(.2105) # convert to uM
N2sat = Henry[2]*(.781)

y = [0, 0, CO2sat*4, 10, 10]
### DIATOM LIGHT ABSORPTION CALCULATIONS.
for k in Iphoto.keys():
    ML.reactions.get_by_id(k).upper_bound = 0
    ML.reactions.get_by_id(k).lower_bound = Iphoto[k]*-1.
    ML.reactions.get_by_id(k).upper_bound = Iphoto[k]*-0.9999

### DIATOM LIMITING RESOURCES CALCULATIONS. 
# Defaults for co2 exchange are ub = 0, lb = 0. Unbounded for now.
ML.reactions.EX_co2_e.upper_bound = 1000
ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,5)
hco3 = 53.70*CO2sat # buffer calculation for hco3, assuming pH = 8.1 with pKa = 6.37 [10^(8.1-6.37) = 53.70]
chla = 0.192e-9
DW = 16.62883198e-12
n = 1.9
#ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3)) # Trimborn, 2009
ML.reactions.EX_hco3_e.lower_bound = 0
ML.reactions.EX_no3_e.lower_bound = -((1.474*y[3])/(6.14+y[3]))

if np.isnan(-((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))):
    pass
else:    
    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))

### CYANOBACTERIA CALCULATIONS

# Remove author's biomass constraint forcing a growth rate at or above observed.
# If this biomass constraint is used, all solutions at low light are infeasible.
ML.reactions.biomass_eq_33047__hc.lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(y[2],O2sat,5)
#ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0

# Nitrogen fixation, including boost by being symbiotic with diatom...
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

ML.reactions.get_by_id("EX_chryso_e").lower_bound = 0

refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(y[2],O2sat,5)
refmodel.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

#ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = -1000 

In [ ]:
def CarbonTransfer(x, T, NO3, Sili):

    Henry = henry(T)
    CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
    O2sat = Henry[1]*(.2105) # convert to uM
    N2sat = Henry[2]*(.781)

    refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
    refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
    refmodel.reactions.get_by_id("EX_cpd00027[e]").lower_bound = 0
    refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

    refsolution = refmodel.optimize()

    #ML.solver.configuration.verbosity = 3
    ML.solver.configuration.timeout = 5
    ML.solver.configuration.tolerance_feasibility = 1e-5

    ML.reactions.EX_co2_e.upper_bound = 1000
    ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat,O2sat,T)
    hco3 = 53.70*CO2sat
    ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3))

    ML.reactions.EX_no3_e.lower_bound = -((1.474*NO3)/(6.14+NO3))

    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(Sili**n))/((8.1**n)+(Sili**n)))
    
    ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
    ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

    Imax = ML.problem.Constraint(
        (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
        lb=-Itot,
        ub=0)
    ML.add_cons_vars(Imax)
    
    AmmoniaConstraint = ML.problem.Constraint(
        ML.reactions.nh4T.flux_expression,
        lb=-4.5*refsolution.fluxes["EX_cpd00528[e]"],
        ub=-4.5*refsolution.fluxes["EX_cpd00528[e]"])
    ML.add_cons_vars(AmmoniaConstraint)
    
    CarbonConstraint = ML.problem.Constraint(
        ML.reactions.carbonT.flux_expression,
        lb=-(x/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound,
        ub=-(x/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
    ML.add_cons_vars(CarbonConstraint)
    
    solution = ML.optimize()
    
    ML.remove_cons_vars(Imax)
    ML.remove_cons_vars(AmmoniaConstraint)
    ML.remove_cons_vars(CarbonConstraint)

    return (solution.fluxes["DM_biomass_c"]-solution.fluxes["biomass_eq_33047__vc"])

In [ ]:
Tn = np.linspace(0,8,8)
NO3 = np.linspace(0,0.125,2)
Sili = np.linspace(0.3,1.5,5)

datamin = np.zeros((len(Tn),len(NO3),len(Sili)))

for idx, temp in enumerate(Tn):
                   
    for idy, NO3ele in enumerate(NO3):

        for idz, Siliele in enumerate(Sili):
            resbrute = []
            counter = 0

            linearrange = np.linspace(0,5,150)
            #linearrange = np.linspace(4-(idx*(0.2))-((5-idz)*(0.6)),5-(idx*(0.125))-((4-idz)*(0.6)),150+(idx))
                
            for idw, C in enumerate(linearrange):
                
                sol = CarbonTransfer(C,temp,NO3ele,Siliele)
                resbrute.append(abs(sol))
    
                print("Temperature:", idx+1)
                print("NO3:", idy+1)
                print("Silicon:", idz+1)
                print("Carbon Iteration:", idw+1)
                print(sol)
                clear_output(wait=True)
            
                counter += 1
    
            index = resbrute.index(min(resbrute))
            datamin[idx,idy,idz] = linearrange[index]

In [ ]:
Tn = np.linspace(0,8,8)
NO3 = np.linspace(0,0.125,2)
Sili = np.linspace(0.3,1.5,5)

datamin = np.zeros((len(Tn),len(NO3),len(Sili)))

for idx, temp in enumerate(Tn):
                   
    for idy, NO3ele in enumerate(NO3):

        for idz, Siliele in enumerate(Sili):
            resbrute = []
            counter = 0

            linearrange = np.linspace(0,5,150)
            #linearrange = np.linspace(4-(idx*(0.2))-((5-idz)*(0.6)),5-(idx*(0.125))-((4-idz)*(0.6)),150+(idx))
                
            for idw, C in enumerate(linearrange):
                
                sol = CarbonTransfer(C,temp,NO3ele,Siliele)
                resbrute.append(abs(sol))
    
                print("Temperature:", idx+1)
                print("NO3:", idy+1)
                print("Silicon:", idz+1)
                print("Carbon Iteration:", idw+1)
                print(sol)
                clear_output(wait=True)
            
                counter += 1
    
            index = resbrute.index(min(resbrute))
            datamin[idx,idy,idz] = linearrange[index]

In [14]:
Tn = np.linspace(-3,26,30)
NO3 = np.linspace(0,.5,3)
Sili = np.linspace(0.2,2,18)

datamin = np.zeros((len(Tn),len(NO3),len(Sili)))

for idx, temp in enumerate(Tn):
                   
    for idy, NO3ele in enumerate(NO3):

        for idz, Siliele in enumerate(Sili):
                
            sol = CarbonTransferAbs(temp,NO3ele,Siliele)
    
            print("Temperature:", Tn[idx])
            print("NO3:", idy)
            print("Si(OH)4:", idz)
            print(sol)
            clear_output(wait=True)
    
            datamin[idx,idy,idz] = sol

0.4773440496107643
-0.4470482284359538
-6.000840955740129
0.07828716320116638
0.07828716320116638
-0.4470482284359538
Temperature: 26.0
NO3: 2
Si(OH)4: 17
6.406611447012825


In [15]:
datamin_re = datamin.reshape(datamin.shape[0], -1)

In [16]:
np.savetxt("CarbonTransfer.csv",datamin_re,delimiter=',')

In [17]:
datamin = datamin_re.reshape(datamin_re.shape[0], datamin_re.shape[1] // len(Sili), len(Sili))

In [ ]:
INSi = RegularGridInterpolator((Tn,NO3,Sili),datamin)
IYSi = RegularGridInterpolator((Tn,NO3),datamin)
ITemp = np.interp(4.5,Tn,datamin[:,-1])

In [ ]:
IAll([5,0.25])

In [ ]:
# Calculate the saturation concentration of carbon dioxide (in mM).
Henry = henry(5)
CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
O2sat = Henry[1]*(.2105) # convert to uM
N2sat = Henry[2]*(.781)

y = [0, 0, CO2sat*4, 10, 10]
### DIATOM LIGHT ABSORPTION CALCULATIONS.
for k in Iphoto.keys():
    ML.reactions.get_by_id(k).upper_bound = 0
    ML.reactions.get_by_id(k).lower_bound = Iphoto[k]*-1.
    ML.reactions.get_by_id(k).upper_bound = Iphoto[k]*-0.9999

# Run this cell if you wish to perform autotrophic growth only.
ML.reactions.EX_glc__D_e.lower_bound = 0
ML.reactions.EX_glc__D_e.upper_bound = 0

# Shut down or turn on the chrysolaminarin reaction.
ML.reactions.EX_chryso_e.lower_bound = 0

ML.reactions.EX_no2_e.lower_bound = 0
ML.reactions.EX_nh4_e.lower_bound = 0
ML.reactions.EX_pi_e.lower_bound = -1
ML.reactions.EX_so4_e.lower_bound = -1
ML.reactions.EX_cncbl3_e.lower_bound = -1

### DIATOM LIMITING RESOURCES CALCULATIONS. 
# Defaults for co2 exchange are ub = 0, lb = 0. Unbounded for now.
ML.reactions.EX_co2_e.upper_bound = 1000
ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat,O2sat,5)
hco3 = 53.70*CO2sat # buffer calculation for hco3, assuming pH = 8.1 with pKa = 6.37 [10^(8.1-6.37) = 53.70]
chla = 0.192e-9
DW = 16.62883198e-12
n = 1.9
ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3)) # Trimborn, 2009
ML.reactions.EX_no3_e.lower_bound = -((1.474*y[3])/(6.14+y[3]))

if np.isnan(-((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))):
    pass
else:    
    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))

### CYANOBACTERIA CALCULATIONS

# Remove author's biomass constraint forcing a growth rate at or above observed.
# If this biomass constraint is used, all solutions at low light are infeasible.
ML.reactions.biomass_eq_33047__hc.lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(y[2],O2sat,5)
#ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0

# Nitrogen fixation, including boost by being symbiotic with diatom...
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

In [10]:
T = 5
NO3 = 9
Sili = 4
Henry = henry(T)
CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
O2sat = Henry[1]*(.2105) # convert to uM
N2sat = Henry[2]*(.781)

n = 1.9

refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
refmodel.reactions.get_by_id("EX_cpd00027[e]").lower_bound = 0
refmodel.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

refsolution = refmodel.optimize()

#    ML.solver.configuration.verbosity = 3
ML.solver.configuration.timeout = 5
ML.solver.configuration.tolerance_feasibility = 1e-5

ML.reactions.EX_co2_e.upper_bound = 1000
ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,T)
hco3 = 53.70*CO2sat
ML.reactions.EX_hco3_e.lower_bound = 0
#    ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3))

ML.reactions.EX_no3_e.lower_bound = -((1.474*NO3)/(6.14+NO3))

ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(Sili**n))/((8.1**n)+(Sili**n)))

ML.reactions.get_by_id("EX_cpd00011[e]").upper_bound = 1000
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
ML.reactions.get_by_id("EX_cpd00011[e]").upper_bound = -RLPC(CO2sat*4,O2sat,T)*0.9999
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
ML.reactions.EX_nh4_e.lower_bound = 0
ML.reactions.EX_nh4_e.upper_bound = 0

Imax = ML.problem.Constraint(
    (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
    lb=-Itot,
    ub=0)
ML.add_cons_vars(Imax)

AmmoniaConstraint = ML.problem.Constraint(
    ML.reactions.nh4T.flux_expression,
    lb=-4.5*refsolution.fluxes["EX_cpd00528[e]"],
    ub=-10*refsolution.fluxes["EX_cpd00528[e]"])
ML.add_cons_vars(AmmoniaConstraint)

BioConstraint = ML.problem.Constraint(
    (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
    lb=0,
    ub=0)
ML.add_cons_vars(BioConstraint)

solution = ML.optimize()

ML.remove_cons_vars(Imax)
ML.remove_cons_vars(AmmoniaConstraint)
ML.remove_cons_vars(BioConstraint)
# ML.remove_cons_vars(NonTrivial)
print("Transferred Glucose: ",solution.fluxes["carbonT"])
print("Cyanobacteria CO2 Intake: ",ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
print("Diatom CO2 Intake: ",ML.reactions.EX_co2_e.lower_bound)
print("Diatom Biomass Flux: ",solution.fluxes["DM_biomass_c"])
print("Cyanobacteria Biomass Flux: ",solution.fluxes["biomass_eq_33047__vc"])
print("Saved Carbon Transfer Value: ",(6*solution.fluxes["carbonT"]/(-ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)))
print("Cyanobacteria Metabolism %: ",100*((solution.fluxes["carbonT"]/((-solution.fluxes["EX_cpd00011[e]"]/6)+solution.fluxes["carbonT"]))))
print("Cyanobacteria Nitrogen %: ",100*((solution.fluxes["nh4T"]-refsolution.fluxes["EX_cpd00528[e]"])/(-refsolution.fluxes["EX_cpd00528[e]"])))

Transferred Glucose:  0.05733972969373664
Cyanobacteria CO2 Intake:  -0.2888378085860543
Diatom CO2 Intake:  -0.9624207826304703
Diatom Biomass Flux:  0.01496357521152003
Cyanobacteria Biomass Flux:  0.01496357521152003
Saved Carbon Transfer Value:  1.191112686550935
Cyanobacteria Metabolism %:  54.361087581756664
Cyanobacteria Nitrogen %:  550.0000000000022
